In [1]:
# Copyright (c) 2024, InfinityQ Technology Inc.

import numpy as np
from titanq import Model, Vtype, Target, S3Storage
from utils import parse_input

# Setting Credentials
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key, and AWS bucket name.

In [2]:
# Enter your API key here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Access Key"
# AWS_SECRET_ACCESS_KEY = "Secret Access Key"
# AWS_BUCKET_NAME = "Bucket Name"

# Setting Up the Problem
Here, we select an instance from the `instances` directory. We then parse the adjacency matrix $A$ and the number of variables $N$ for the instance's graph.

In [3]:
instance_file_name = 'C125.9.clq'
instance_folder_path = 'instances'

A, N = parse_input(f'{instance_folder_path}/{instance_file_name}')

In [4]:
# Display adjacency matrix from above
print(f'Number of vertices: {N=}')
print('-------- Adjacency Matrix --------')
print(A)
print('----------------------------------')

Number of vertices: N=125
-------- Adjacency Matrix --------
[[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 0. ... 1. 1. 1.]
 [1. 0. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]]
----------------------------------


# Construct the Objective and Constraints for TitanQ
Recall that our objective is to maximize the size of our clique. Our clique is represented by a binary vector $\mathbf{x}=\begin{bmatrix}x_1 & x_2 & \cdots & x_N\end{bmatrix}^T \in \{0, 1\}^N$ where $x_i = 1$ when the $i^{th}$ node is in the clique. The size of the clique is then $\sum_{i=1}^{N}x_i=\begin{bmatrix}1 & 1 & \cdots & 1\end{bmatrix}\mathbf{x}$. To formulate this in TitanQ, we can take our bias vector as $b=\begin{bmatrix}1 & 1 & \cdots & 1\end{bmatrix}^T$. Note that to frame this as a minimization problem, we must negate $b$ so that our objective function is minimized at the maximum clique size.

In [5]:
weights = None
bias = -np.ones((N,), dtype=np.float32)

Additionally, as discussed in the [README](README.md), we have our quadratic constraint that asserts that the set defined by $x$ is indeed a clique. This constraint is of the form $\mathbf{x}^T(\mathbf{1}_{N\times N}-\mathbf{A}-\mathbf{I})x=0$, and can be formulated in TitanQ as follows.


In [6]:
quadratic_constraint_mask = np.ones((N,N), dtype=np.float32) - A -  np.eye(N, dtype=np.float32)
quadratic_constraint_limit = 0

# Inputting the Problem to TitanQ
The user should provide the credentials they defined earlier. We then initialize a TitanQ model by creating the variable $x$ and setting the weights and bias vector.

In [7]:
##############
# TitanQ SDK #
##############
model = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)
model.add_variable_vector(name = 'x', size = N, vtype = Vtype.BINARY)
model.set_objective_matrices(weights, bias, target = Target.MINIMIZE)
model.add_quadratic_equality_constraint(quadratic_constraint_mask, quadratic_constraint_limit)

The user can modify the below parameters to tune the problem for a specific instance. Increasing the timeout results in the solver running for longer, allowing it to find better quality solutions. To tune the number of chains and engines, as well as the beta values, see the [tuning guide](https://docs.titanq.infinityq.io/user-guide/parameter_tuning_guide).

In [8]:
timeout_in_seconds = 10
num_chains = 32
num_engines = 8
T_min = 1e-4
T_max = 1

titanq_args = {
  'timeout_in_secs': timeout_in_seconds,
  'num_chains': num_chains,
  'num_engines': num_engines,
  'beta': list(1/np.geomspace(T_min, T_max, num_chains)),
}

In [9]:
response = model.optimize(**titanq_args)

# Printing the Results
TitanQ has solved the model and found a potential maximum clique on each engine. We can check if the solution found corresponds to a clique by checking if our quadratic constraint was violated. Here, we print the result of each engine, as well as the best found valid result.

In [10]:
best_idx = -1
best_size = 0
max_clique = []

print('-------- All Engine Results --------')
for engine, (neg_size, solution) in enumerate(response.result_items()):
    clique = [i for i, state in enumerate(solution) if state != 0]
    is_clique = response.constraint_violations()[0][engine] == 0
    if is_clique and -neg_size > best_size:
        best_idx = engine
        best_size = -neg_size
        max_clique = clique
    print(f'\n---- Engine {engine + 1} ----')
    print(f'\tClique Size = {-neg_size}\t Clique = {clique}')
    print(f'\t which is {"a clique" if is_clique else "not a clique"}.')

-------- All Engine Results --------

---- Engine 1 ----
	Clique Size = 34	 Clique = [4, 8, 10, 13, 16, 18, 23, 24, 28, 30, 33, 39, 43, 44, 48, 51, 53, 54, 64, 65, 66, 69, 76, 78, 79, 95, 97, 98, 102, 103, 109, 116, 121, 124]
	 which is a clique.

---- Engine 2 ----
	Clique Size = 34	 Clique = [0, 1, 4, 6, 8, 10, 17, 24, 28, 30, 33, 39, 43, 44, 47, 48, 53, 59, 67, 69, 70, 76, 78, 79, 98, 100, 109, 113, 114, 116, 120, 121, 122, 124]
	 which is a clique.

---- Engine 3 ----
	Clique Size = 34	 Clique = [0, 4, 6, 8, 10, 12, 16, 18, 23, 24, 28, 30, 33, 43, 44, 48, 51, 53, 54, 65, 69, 76, 78, 79, 84, 95, 97, 98, 102, 103, 109, 116, 121, 124]
	 which is a clique.

---- Engine 4 ----
	Clique Size = 34	 Clique = [0, 4, 6, 8, 10, 16, 18, 24, 28, 30, 33, 43, 44, 48, 51, 54, 65, 69, 76, 78, 79, 81, 84, 90, 95, 97, 102, 103, 109, 113, 116, 120, 121, 124]
	 which is a clique.

---- Engine 5 ----
	Clique Size = 34	 Clique = [4, 6, 8, 10, 16, 18, 23, 24, 28, 30, 33, 39, 43, 48, 51, 53, 54, 64, 65, 66,

In [11]:
print('\n-------- Best Solution --------')
if best_idx == -1:
    print("None of the engines returned a valid solutuon.")
    print("Try adjusting the hyperparameters further to yield valid solutions.")
else:
    print(f"---- Engine {best_idx + 1} ----")
    print(f'\tClique Size = {best_size}\t Clique = {max_clique}')


-------- Best Solution --------
---- Engine 1 ----
	Clique Size = 34	 Clique = [4, 8, 10, 13, 16, 18, 23, 24, 28, 30, 33, 39, 43, 44, 48, 51, 53, 54, 64, 65, 66, 69, 76, 78, 79, 95, 97, 98, 102, 103, 109, 116, 121, 124]
